# Introduction
This Google Colab notebook aims to implement and train a Generative Adversarial Network (GAN) to generate surrealist art.
The GAN is constructed using a two-part architecture: a Generator model that synthesizes new images from random noise, and a Discriminator model that evaluates whether an image is real or generated. By training on a dataset of surrealist art images, the GAN learns to generate novel images in a similar style, making it an exciting application of deep learning in the field of art generation.



# Purpose
### The primary objectives of this project are:

* **Preprocessing:** Load and preprocess a dataset of surrealist art images into a suitable format for training.
* **GAN Architecture:** Build and define both Generator and Discriminator models, which form the core components of the GAN architecture.
* **Training:** Train the GAN on the surrealist art dataset, utilizing a WGAN-GP (Wasserstein GAN with Gradient Penalty) approach to enhance training stability.
* **Evaluation:** Track training progress through loss metrics and visualize generated images at various stages to assess the GAN's output quality.
* **Saving Models:** Save the trained Generator and Discriminator models to Google Drive for future use or further refinement.




Overall, this project demonstrates how deep learning can be applied in the realm of art generation, producing new images inspired by surrealist artwork. By the end of this notebook, you will have a trained GAN capable of generating surrealist-style images, along with models saved for future use or modification.

# Imports & Mount Google Drive
This section mounts Google Drive to access datasets and save results directly to it:

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import colors, pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
import h5py  # for saving to HDF5 format
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import distutils as _distutils
import importlib
import inspect as _inspect

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Paths and Preprocessing Functions
Defines the paths to the necessary directories and includes functions to preprocess art images:

In [ ]:
# Define paths
spectrogram_dir = '/content/drive/MyDrive/COSC_5470/spectrograms' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/spectrograms'
art_dataset_dir = '/content/drive/MyDrive/COSC_5470/AI_Abstract_Surrealism_DataSet' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/AI_LD_surrealism'
label_csv_path = '/content/drive/MyDrive/COSC_5470/song_metadata.csv' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/song_metadata.csv'

In [ ]:
def preprocess_art_images(art_dataset_dir, target_size=(256, 256)):
    images = []
    for img_file in os.listdir(art_dataset_dir):
        img_path = os.path.join(art_dataset_dir, img_file)

        # Check if the file is a valid image
        try:
            img = load_img(img_path, target_size=target_size)
            img = img_to_array(img)

            # Normalize the image to [-1, 1]
            img = (img - 127.5) / 127.5

            images.append(img)
        except Exception as e:
            print(f"Skipping file {img_file} due to error: {e}")

    return np.array(images)

# Loading and Saving Preprocessed Images
Loads and preprocesses art images, and optionally saves them for later use:

In [ ]:
# Load and preprocess images before training
art_images = preprocess_art_images(art_dataset_dir, target_size=(256, 256))

In [ ]:
np.save('processed_art_images.npy', art_images)  # Save for later use or immediate training

# Visualization Function
Provides a utility function to visualize a selection of preprocessed images:

In [ ]:
# Function to visualize data
def visualize_data(images, n=5):
    plt.figure(figsize=(10, 10))
    for i in range(n):
        ax = plt.subplot(1, n, i + 1)
        img = 0.5 * images[np.random.randint(0, len(images))] + 0.5  # Rescale images to [0, 1]
        plt.imshow(img)
        plt.axis('off')
    plt.show()

In [ ]:
visualize_data(art_images)

# Loading and Saving Preprocessed Images
Loads and preprocesses art images, and optionally saves them for later use:

In [ ]:
# Build Generator Model
def build_generator(latent_dim):
    generator_input = Input(shape=(latent_dim,))
    x = Dense(128 * 64 * 64)(generator_input)
    x = LeakyReLU()(x)
    x = Reshape((64, 64, 128))(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(128, 5, strides=2, padding='same')(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64, 5, strides=2, padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(3, 7, activation='tanh', padding='same')(x)
    generator = Model(inputs=generator_input, outputs=x)
    return generator


# Build Discriminator Model
def build_discriminator(image_shape):
    discriminator_input = Input(shape=image_shape)
    x = Conv2D(64, 5, strides=2, padding='same')(discriminator_input)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = Conv2D(128, 5, strides=2, padding='same')(x)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = Conv2D(256, 5, strides=2, padding='same')(x)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    discriminator = Model(inputs=discriminator_input, outputs=x)
    return discriminator

# Gradient Penalty
Defines a function to compute the gradient penalty for training:

In [ ]:
# Function to compute gradient penalty
def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    alpha = np.random.random((real_samples.shape[0], 1, 1, 1))
    interpolates = alpha * real_samples + (1 - alpha) * fake_samples
    with tf.GradientTape() as tape:
        tape.watch(interpolates)
        validity_interpolates = discriminator(interpolates, training=True)
    gradients = tape.gradient(validity_interpolates, [interpolates])[0]
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))
    gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2)
    return gradient_penalty


# Training the GAN
Includes the main training loop for the GAN:

In [ ]:
# GAN Training Loop
# Training Function
def train_gan(generator, discriminator, generator_optimizer, discriminator_optimizer, images, epochs, batch_size, latent_dim, lambda_gp, scheduler):
    real_label = -1  # Labels for the "real" and "fake" samples
    fake_label = 1
    for epoch in range(epochs):
        for _ in range(images.shape[0] // batch_size):
            # Training Discriminator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            noise = tf.convert_to_tensor(noise, dtype=tf.float32)  # Convert noise to tensor
            real_imgs = images[np.random.randint(0, images.shape[0], batch_size)]
            real_imgs = tf.convert_to_tensor(real_imgs, dtype=tf.float32)  # Convert real images to tensor
            gen_imgs = generator(noise, training=True)

            with tf.GradientTape() as tape:
                real_loss = discriminator(real_imgs, training=True)
                fake_loss = discriminator(gen_imgs, training=True)
                gp = compute_gradient_penalty(discriminator, real_imgs, gen_imgs)
                d_loss = fake_loss - real_loss + lambda_gp * gp
            d_gradients = tape.gradient(d_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))

            # Training Generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            noise = tf.convert_to_tensor(noise, dtype=tf.float32)  # Convert noise to tensor again for generator
            with tf.GradientTape() as tape:
                gen_imgs = generator(noise, training=True)
                g_loss = -discriminator(gen_imgs, training=True)
            g_gradients = tape.gradient(g_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))

        # Output training progress
        print(f"Epoch {epoch + 1}/{epochs}, D Loss: {tf.reduce_mean(d_loss)}, G Loss: {tf.reduce_mean(g_loss)}")
        if (epoch + 1) % 10 == 0:
            visualize_data(gen_imgs.numpy(), n=5)  # Visualize generated images every 10 epochs



# Saving Generated Images
Defines a function to save generated images from the generator:

In [ ]:
def save_images(epoch, generator, latent_dim, image_save_dir="saved_images"):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale images 0 - 1
    os.makedirs(image_save_dir, exist_ok=True)
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"{image_save_dir}/epoch_{epoch}.png")
    plt.close()

# Setting Up and Training the GAN
Sets hyperparameters, initializes models, and begins training:

In [ ]:
# Set hyperparameters and paths
latent_dim = 100
epochs = 100
batch_size = 16
lambda_gp = 10
art_dataset_dir = '/content/drive/MyDrive/COSC_5470/AI_Abstract_Surrealism_DataSet' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/AI_LD_surrealism'
images = preprocess_art_images(art_dataset_dir)


Skipping file .DS_Store due to error: cannot identify image file <_io.BytesIO object at 0x7cbaa71c1c60>


In [ ]:
# Initialize models and optimizers
generator = build_generator(latent_dim)
discriminator = build_discriminator((256, 256, 3))
generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)


In [ ]:
train_gan(generator, discriminator, generator_optimizer, discriminator_optimizer, images, epochs, batch_size, latent_dim, lambda_gp, None)


# Saving the Models
Saves the trained models to Google Drive for future use:

In [ ]:
# Save the trained models
generator.save('/content/drive/My Drive/COSC_5470/saved_generator_model_final.keras')
discriminator.save('/content/drive/My Drive/COSC_5470/saved_discriminator_model_final.keras')
# gan.save('/content/drive/My Drive/COSC_5470/saved_gan_model_final.keras')

# Visualization of GAN Training Metrics
This section of the notebook visualizes GAN training metrics extracted from a log file. The log file contains information about each training epoch, including the losses for both the Discriminator and Generator models, which are key metrics for evaluating the GAN's training progress.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd

# Read the data from the uploaded file
file_path = '/content/drive/My Drive/COSC_5470/training_log1.txt'  # Ensure the path is correctly specified

# Initialize lists to store the extracted data
epochs = []
d_losses = []
g_losses = []

with open(file_path, 'r', encoding='utf-8-sig') as file:  # 'utf-8-sig' removes the BOM if present
    for line in file:
        if 'Epoch' in line:
            parts = line.split(',')
            epoch = int(parts[0].split(' ')[1].split('/')[0])  # Parsing the epoch number
            d_loss = float(parts[1].split(': ')[1])  # Parsing the discriminator loss
            g_loss = float(parts[2].split(': ')[1])  # Parsing the generator loss

            epochs.append(epoch)
            d_losses.append(d_loss)
            g_losses.append(g_loss)

# Create a DataFrame
data = pd.DataFrame({
    'Epoch': epochs,
    'Discriminator Loss': d_losses,
    'Generator Loss': g_losses
})

# Plotting with Matplotlib and Seaborn
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
sns.lineplot(x='Epoch', y='Discriminator Loss', data=data, label='Discriminator Loss')
sns.lineplot(x='Epoch', y='Generator Loss', data=data, label='Generator Loss')
plt.title('Loss During Training')
plt.grid(True)

plt.subplot(1, 2, 2)
sns.lineplot(x='Epoch', y='Generator Loss', data=data)
plt.title('Generator Loss During Training')
plt.grid(True)
plt.show()

# Interactive Plotting with Plotly
fig = px.line(data, x='Epoch', y=['Discriminator Loss', 'Generator Loss'],
              title='Interactive Training Metrics Visualization', labels={'value': 'Metric Value', 'variable': 'Metrics'})
fig.show()




---

